<a href="https://colab.research.google.com/github/schwartz-cnl/Computational-Neuroscience-Class/blob/main/Perceptron/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perceptron

This is a simple example illustrating the classic perceptron algorithm. A linear decision function parametrized by the weight vector "w" and bias paramter "b" is learned by making small adjustements to these parameters every time the predicted label "f_i" mismatches the true label "y_i" of an input data point "x_i".

The predicted label corresponds to the following function:

             f_i = sign( < w, x_i> + b),

where "< . , . >"  denotes the dot product operation.
The update rule is given as follows:

if "y_i != f_i" (predicted label f_i different from true label y_i) then

          w <-- w + y_i*x_i; and  
          b <-- b + y_i;

else continue with next data sample The above process is repeated over the set of samples several times.

If data points are linearly separable the above rule is guaranteed to converge in a finite number of iterations. Proof see: http://www.cs.columbia.edu/~mcollins/courses/6998-2012/notes/perc.converge.pdf


2016 Luis G Sanchez Giraldo and Odelia Schwartz. Transcribed and modified to Python by Xu Pan, 2022.

In [ ]:
# Construct a simple data set based on MNIST images
# This is a data set of handwritten digits 0 to 9

# Download MNIST dataset from keras

from keras.datasets import mnist
import numpy as np

(train_X, train_y), (test_X, test_y) = mnist.load_data()
print(test_X.shape)
print(test_y.shape)

In [ ]:
# y are the digit labels
print(train_y[0:10])

In [ ]:
# Sort dataset by label.

sorted_test_X =[[], [], [], [], [], [], [], [], [], []]
for i, y in enumerate(test_y):
  sorted_test_X[y].append(test_X[i,:,:])

In [ ]:
# Create a simple two-class problem using images of digits 0 and 5 from
# the MNIST test data set  
pos_class = 0 # 3
neg_class = 5

In [ ]:
# get samples from positive and negative classes 
pos_data = np.array(sorted_test_X[pos_class])
neg_data = np.array(sorted_test_X[neg_class])
print(pos_data.shape)
print(neg_data.shape)

In [ ]:
# Look at some digits from the classes
# Look at different samples from each class (here plotted just the first; try others)

import matplotlib.pyplot as plt

plt.imshow(pos_data[0,:,:], cmap='binary')
plt.show()
plt.imshow(neg_data[0,:,:], cmap='binary')
plt.show()

In [ ]:
# Gather the samples from the two classes into one matrix X
# Note that there the samples from each class are appended 
# to make up 1872 samples altogether

X = np.concatenate((pos_data, neg_data), axis=0)/255
X = np.reshape(X, (X.shape[0],-1))
print(X.shape)

In [ ]:
# Label the two classes with 1 and -1 respectively
Y = np.concatenate((np.ones(pos_data.shape[0]), -np.ones(neg_data.shape[0])), axis=0);
print(Y.shape)
print(Y[0:10])
print(Y[981:991])

In [ ]:
# Choose random samples from data. To do so:
# permute data samples to run the learning  algorithm 
# and take just n_samples from the permuted data (here 60 samples)

n_samples = 60

p_idx = np.random.permutation(X.shape[0])
X = X[p_idx[0:n_samples], :]
Y = Y[p_idx[0:n_samples]]

In [ ]:
# Project the data onto the means of the two classes
# First look at the mean of the two class

plt.imshow(np.reshape(np.mean(X[Y == 1, :], axis=0),(28,28)), cmap='binary')
plt.show()

# mean of second class
plt.imshow(np.reshape(np.mean(X[Y == -1, :], axis=0),(28,28)), cmap='binary')
plt.show()


In [ ]:
# Now project the data

V = np.stack((np.mean(X[Y == 1, :], axis=0), np.mean(X[Y == -1, :], axis=0)), axis=1)
V[:, 0] = V[:, 0]/np.linalg.norm(V[:, 0])
V[:, 1] = V[:, 1]/np.linalg.norm(V[:, 1])

Z = np.matmul(X,V)
print(Z.shape)
plt.figure(figsize=(5,5))
plt.scatter(Z[Y == 1,0], Z[Y == 1,1], c='g')
plt.scatter(Z[Y == -1,0], Z[Y == -1,1], c='r')

In [ ]:
# This is a helper function that plots the partition.

def display2DPartition(Z, Y, w, b):
  plt.figure(figsize=(5,5))
  ax = plt.axes()
  plt.scatter(Z[Y == 1,0], Z[Y == 1,1], c='g', zorder=1)
  plt.scatter(Z[Y == -1,0], Z[Y == -1,1], c='r', zorder=2)
  ax.set_facecolor('pink')
  x = ax.get_xlim()
  y = ax.get_ylim()
  y_line = -(np.array(x)*w[0]+b)/w[1]
  plt.fill_between(x,y_line, color='palegreen',zorder=0)
  for iSmp in range(Z.shape[0]):
    z_i = Z[iSmp, :]
    # compute prediction with current w and b
    f_i = np.sign(np.dot(w, z_i) + b)
    if f_i != Y[iSmp]:
      plt.scatter(z_i[0], z_i[1], marker='o', s=100, facecolors='none',linewidths=2, edgecolors='black', zorder=3)
  ax.set_xlim(x)
  ax.set_ylim(y)

In [ ]:
# Simple Learning algorithm for Perceptron
# here we denote two classes: the positive class by label "1"  and the negative
# class by label "-1." 
# Any point in the plane colored as green will be classified as positive
# class and any point falling within the red region as negative class.
# Training samples are denoted by the green crosses (positive) and red dots
# (negative). A missclassified training point, that is "f_i != y_i" is 
# marked with a circle 

from IPython.display import clear_output
import time

# first initialize the parameters
lr = 1                           # Learning rate parameter (1 in the classic perceptron algorithm)
w = np.random.randn(Z.shape[1])  # Initial guess for the hyperplane parameters
print(w)
b = 0                            # bias is initially zero
print(b)
max_epoch = 100                  # Number of epoch (complete loops trough all data)
epoch = 1                        # epoch counter

# display the starting decision hyhyperplane (partition of the space)
# (compare this to the decision hyperplane that is learned!)
display2DPartition(Z, Y, w, b)
plt.show()


In [ ]:
# now run the perceptron training
while epoch <= max_epoch:
  # loop trough all data points one time (an epoch)
  for iSmp in range(n_samples):
    z_i = Z[iSmp, :]
    # compute prediction with current w and b
    f_i = np.sign(np.dot(w, z_i) + b)
    # update w and b if missclassified
    if f_i != Y[iSmp]:
      # TODO: update the w and b according to the equation.
      # Note that the notation here is different from the introduction.
      w =
      b =
  # diplay current decision hyperplane (partition of the space)
  clear_output(wait=True)
  display2DPartition(Z, Y, w, b)
  plt.show()
  time.sleep(0.1)
  epoch = epoch + 1;

In [ ]:
# display the learned decision hyperplane (partition of the space)
print(w)
print(b)
display2DPartition(Z, Y, w, b)
plt.show()

###Exercise: 
1) Complete the training code to update w and b.

2) After trying the code for the given classes,
try running the code again, but this time changing the digits of the
positive or negative class. 
You can do this by changing the following two lines above:

pos_class = 0

neg_class = 5

What classes are easier to learn?
